In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme.active_learning import *
from aideme.initial_sampling import *
from aideme.explore import *
from aideme.utils.metrics import three_set_metric, classification_metrics
from aideme.utils.plotting import plot_polytope
from aideme.utils.convergence import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)
labeled_set = LabeledSet(y, partial=y_subspace)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [6]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = float('inf')
INITIAL_SAMPLER = StratifiedSampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = RandomInitialSampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002905488606333965, tsm: 0.0
iter: 11, fscore: 0.044043624161073824, tsm: 0.0
iter: 21, fscore: 0.09251101321585903, tsm: 0.0
iter: 31, fscore: 0.05555555555555555, tsm: 3.8527232332053375e-05
iter: 41, fscore: 0.7790697674418605, tsm: 0.00022459292532285232
iter: 51, fscore: 0.5416666666666666, tsm: 0.006538139145012573
iter: 61, fscore: 0.8306451612903226, tsm: 0.024880382775119617
iter: 71, fscore: 0.7719298245614035, tsm: 0.2703862660944206
iter: 81, fscore: 0.900523560209424, tsm: 0.4425287356321839
iter: 91, fscore: 0.9292929292929293, tsm: 0.674074074074074
iter: 101, fscore: 0.9552238805970149, tsm: 0.8275862068965517


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
0,begin,0.005163,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.039544,"[70241, 71261]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.001104,0.038440,0.002905,0.000000
2,exploration,0.034911,[92283],[0.0],"[[0.0, 0.0]]",0.000735,0.034175,NaN,NaN
3,exploration,0.033511,[2240],[0.0],"[[0.0, 0.0]]",0.000459,0.033052,NaN,NaN
4,exploration,0.033282,[32465],[0.0],"[[0.0, 0.0]]",0.000518,0.032764,NaN,NaN
...,...,...,...,...,...,...,...,...,...
97,exploration,0.014400,[35330],[1.0],"[[1.0, 1.0]]",0.012558,0.001843,NaN,NaN
98,exploration,0.005112,[53711],[0.0],"[[1.0, 0.0]]",0.003467,0.001645,NaN,NaN
99,exploration,0.004499,[21848],[0.0],"[[1.0, 0.0]]",0.002800,0.001699,NaN,NaN
100,exploration,0.006684,[33195],[0.0],"[[1.0, 0.0]]",0.004838,0.001846,NaN,NaN


# FACTORIZATION EXAMPLE

In [5]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = float('inf')
INITIAL_SAMPLER = StratifiedSampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = RandomInitialSampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode='positive', sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.008114687584527996, tsm: 0.0
iter: 11, fscore: 0.2702702702702703, tsm: 0.0
iter: 21, fscore: 0.5314685314685315, tsm: 0.2111111111111111
iter: 31, fscore: 0.9585253456221198, tsm: 0.5982905982905983
iter: 41, fscore: 0.9230769230769231, tsm: 0.8256880733944955
iter: 51, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
0,begin,0.005836,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.596208,"[75180, 91500]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.494131,0.102077,0.008115,0.000000
2,exploration,0.107773,[47825],[0.0],"[[0.0, 0.0]]",0.101002,0.006771,NaN,NaN
3,exploration,0.397004,[18130],[0.0],"[[0.0, 0.0]]",0.070967,0.326037,NaN,NaN
4,exploration,0.015293,[29863],[0.0],"[[0.0, 0.0]]",0.013014,0.002279,NaN,NaN
5,exploration,0.086419,[99381],[0.0],"[[1.0, 0.0]]",0.012452,0.073967,NaN,NaN
6,exploration,0.044125,[95683],[0.0],"[[0.0, 0.0]]",0.003645,0.040480,NaN,NaN
7,exploration,0.154549,[17541],[0.0],"[[0.0, 0.0]]",0.008293,0.146256,NaN,NaN
8,exploration,0.002713,[10603],[0.0],"[[1.0, 0.0]]",0.001381,0.001331,NaN,NaN
9,exploration,0.005231,[5988],[0.0],"[[0.0, 0.0]]",0.004373,0.000857,NaN,NaN
